## Data Load in

In [ ]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/diffusion_model_mix"
if not os.path.exists(datadir):
  !ln -s "/content/drive/Shareddrives/CS444 DLCV 抱團/Final Project/diffusion model mix/" $datadir # TODO: Fill your Assignment 4 path
os.chdir(datadir)
!pwd

/content/drive/Shareddrives/CS444 DLCV 抱團/Final Project/diffusion model mix


In [ ]:
# 解壓縮
!unzip "/content/drive/Shareddrives/CS444 DLCV 抱團/Final Project/test.zip" -d "/content/data2"


串流輸出內容已截斷至最後 5000 行。
  inflating: /content/data2/test/cc358864-bacb-11e8-b2b8-ac1f6b6435d0_blue.png  
  inflating: /content/data2/__MACOSX/test/._cc358864-bacb-11e8-b2b8-ac1f6b6435d0_blue.png  
  inflating: /content/data2/test/bfd7cda4-bac9-11e8-b2b8-ac1f6b6435d0_yellow.png  
  inflating: /content/data2/__MACOSX/test/._bfd7cda4-bac9-11e8-b2b8-ac1f6b6435d0_yellow.png  
  inflating: /content/data2/test/b029f372-bad9-11e8-b2b9-ac1f6b6435d0_yellow.png  
  inflating: /content/data2/__MACOSX/test/._b029f372-bad9-11e8-b2b9-ac1f6b6435d0_yellow.png  
  inflating: /content/data2/test/7a0c7fd4-bac6-11e8-b2b7-ac1f6b6435d0_yellow.png  
  inflating: /content/data2/__MACOSX/test/._7a0c7fd4-bac6-11e8-b2b7-ac1f6b6435d0_yellow.png  
  inflating: /content/data2/test/404e60b2-bac8-11e8-b2b7-ac1f6b6435d0_green.png  
  inflating: /content/data2/__MACOSX/test/._404e60b2-bac8-11e8-b2b7-ac1f6b6435d0_green.png  
  inflating: /content/data2/test/9fa35cce-bac9-11e8-b2b8-ac1f6b6435d0_green.png  
  inflating: /cont

In [ ]:
import pandas as pd

df = pd.read_csv("/content/data2/sample_submission.csv")
df.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0


In [ ]:

import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

def modify_first_conv(model, in_channels):
    old_conv = model.conv1
    new_conv = nn.Conv2d(in_channels, old_conv.out_channels,
                         kernel_size=old_conv.kernel_size,
                         stride=old_conv.stride,
                         padding=old_conv.padding,
                         bias=old_conv.bias is not None)

    # Copy weights from the first 3 channels
    with torch.no_grad():
        new_conv.weight[:, :3] = old_conv.weight
        if in_channels > 3:
            # Initialize extra channel(s) with zeros
            new_conv.weight[:, 3:] = old_conv.weight[:, :1].mean(dim=1, keepdim=True)

    model.conv1 = new_conv
    return model

In [ ]:
class MixedSupervisionModel(nn.Module):
    def __init__(self, num_classes=28, backbone_type='resnet18', beta_param=1.0, in_channels=4):
        super().__init__()
        self.beta_param = beta_param

        # Load and modify backbone
        if backbone_type == 'resnet18':
            backbone = models.resnet18(pretrained=True)
        elif backbone_type == 'resnet34':
            backbone = models.resnet34(pretrained=True)
        else:
            raise ValueError("backbone_type must be 'resnet18' or 'resnet34'")

        backbone = modify_first_conv(backbone, in_channels=in_channels)
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # remove FC
        self.feature_dim = backbone.fc.in_features

        self.classifier = nn.Linear(self.feature_dim, num_classes)
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, img_real, label_real, img_gen=None, label_gen=None):
        feat_real = self.backbone(img_real).squeeze(-1).squeeze(-1)

        if img_gen is not None and label_gen is not None:
            batch_size = img_real.size(0)


            lam = torch.distributions.Beta(self.beta_param, self.beta_param).sample((batch_size,)).to(img_real.device)
            lam = lam.view(-1, 1)

            feat_gen = self.backbone(img_gen).squeeze(-1).squeeze(-1)


            mixed_feat = lam * feat_real + (1 - lam) * feat_gen
            mixed_label = lam * label_real + (1 - lam) * label_gen
        else:
            mixed_feat = feat_real
            mixed_label = label_real


        logits = self.classifier(mixed_feat)
        loss = self.criterion(logits, mixed_label)
        return loss, logits

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd


checkpoint = torch.load('/content/diffusion_model_mix/mixModel_34.pth', map_location='cuda')  # or 'cpu'

model = MixedSupervisionModel(
    num_classes=28,
    backbone_type='resnet34',
    beta_param=0.3,
    in_channels=4
)
model.load_state_dict(checkpoint['model_state_dict'])


df = pd.read_csv("/content/data2/sample_submission.csv")


transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

colors = ['red', 'green', 'blue', 'yellow']
threshold = 0.5
all_preds = []

for img_id in df['Id']:

    channels = []
    for color in colors:
        img_path = f"/content/data2/test/{img_id}_{color}.png"
        image = Image.open(img_path).convert('L')
        image = transform(image)
        channels.append(image)

    image_4ch = torch.cat(channels, dim=0).unsqueeze(0).to(device)  # (1, 4, H, W)

    with torch.no_grad():
        dummy_label = torch.zeros((1, model.classifier.out_features), device=device)  # shape: (1, 28)
        loss, logits = model(image_4ch, dummy_label)
        probs = torch.sigmoid(logits)
        pred_onehot = (probs > threshold).squeeze(0).cpu().int()  # (C,)


        if pred_onehot.sum() == 0:

            top1 = torch.argmax(probs, dim=1).item()
            pred_onehot[top1] = 1


        pred_indices = [str(i) for i, val in enumerate(pred_onehot) if val == 1]
        pred_str = " ".join(pred_indices)
        all_preds.append(pred_str)



df['Predicted'] = all_preds
df.to_csv("/content/diffusion_model_mix/submission_mixModel_hidden_34.csv", index=False)
print("✅ Submission saved to /content/submission.csv")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 173MB/s]


✅ Submission saved to /content/submission.csv
